<a href="https://colab.research.google.com/github/liam-haas/CLPS0950_FinalProject/blob/main/Project_Testbed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np

A = 'C=CC'
B = 'C=CC=C'

def index_fun(x):
  indices = [index for index in range(len(x)) if x.startswith('=',index)]
  # print(indices) # prints all occurances of double bonds in the SMILES string
  
  for n in range(0,len(indices)):
    a = indices[n]
    if x[a-1] == 'C' and x[a+1] == 'C':
      print('Alkene found at position ', a)
    else:
      continue

  return indices

index_fun(B)

import pandas as pd
# pd.read_csv('Test SMILES sequences - Sheet1.csv')
d = {'IUPAC name':['ethene','prop-1-ene','3-methylpent-1-ene','1-phenylprop-2-ene','but-1,3-diene'],
     'SMILES sequence':['C=C','C=CC','C=CC(CC)C','C=CCC1=CC=CC=C1','C=CC=C']}
d['Alkenes'] = [index_fun(item) for item in d['SMILES sequence']] 
df = pd.DataFrame(data=d)
df

# d.update({'Alkene positions':''})

Alkene found at position  1
Alkene found at position  4
Alkene found at position  1
Alkene found at position  1
Alkene found at position  1
Alkene found at position  1
Alkene found at position  9
Alkene found at position  12
Alkene found at position  1
Alkene found at position  4


,IUPAC name,SMILES sequence,Alkenes
0,ethene,C=C,[1]
1,prop-1-ene,C=CC,[1]
2,3-methylpent-1-ene,C=CC(CC)C,[1]
3,1-phenylprop-2-ene,C=CCC1=CC=CC=C1,"[1, 6, 9, 12]"
4,"but-1,3-diene",C=CC=C,"[1, 4]"
